In [5]:
import numpy as np
import matplotlib.pyplot as plt
from cs231n.layers import *
from cs231n.fast_layers import *
from game_nn import *
from cs231n import optim
import numpy as np
import cPickle as pickle

In [21]:
# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 1 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H,4) / np.sqrt(H)

In [22]:
def simulator(action=None):
    env = np.random.rand(80, 80)
    feedback = np.array([0, 0, 0, 0])
    return env, feedback 

In [23]:
def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

In [24]:
def softmax(x):
    #compute log propbality
    probs = np.exp(x - np.max(x, axis=1, keepdims=True))
    probs /= np.sum(probs, axis=1, keepdims=True)
    return probs

In [59]:
def policy_forward(env):
    env = env.ravel()
    env = env.reshape(-1,1)
    h = np.dot(model['W1'], env) #should be 200 * 1
    h[h<0] = 0 # ReLU nonlinearity
    action_score = np.dot(h.T, model['W2']) # should be (1 , 4)
    action = np.argmax(action_score)
    return action, h, env # return action, and hidden state

# reward should be a vector e.g [0, 0, 1, 0]
def policy_backward(h_cache, env_cache, feedback):
    """ backward pass. (h_cache is array of intermediate hidden states) """
    feedback = feedback.reshape(1,-1)
    dW2 = np.dot(h_cache, feedback) # 200 * 4
#     dh = np.outer(reward, model['W2'])
    dh = np.dot(model['W2'], feedback.T) # 200 * 1
    dh[h_cache <= 0] = 0 # backpro prelu
    #epx ex input (D*D, 1)
    dW1 = np.dot(dh, env_cache.T)
    return {'W1':dW1, 'W2':dW2}

In [60]:
# perform sgd parameter update 
def sgd_update(model, gradient, learning_rate):
    for k,v in model.iteritems():
        model[k] += learning_rate * gradient[k]


In [62]:
# training process
def train_game_nn(model, learning_rate, max_iter = 10):
    env, _ = simulator() # initial env
    for i in range(max_iter):
        action, h_cache, env_cache =policy_forward(env)
        env, feedback = simulator(action)
        gradient = policy_backward(h_cache, env_cache, feedback)
        sgd_update(model, gradient, learning_rate)



In [63]:
train_game_nn(model, learning_rate)